# Google Oauth

In [1]:
import os.path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [6]:
# SCOPES = ["https://www.googleapis.com/auth/calendar"]
# creds = None

# if os.path.exists('token.json'):
#     creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# # 자격 증명이 없거나 만료됐으면 새로 로그인
# if not creds or not creds.valid:
#     if creds and creds.expired and creds.refresh_token:
#         creds.refresh(Request())
#     else:
#         # credentials.json 파일에서 클라이언트 비밀키 불러오기
#         flow = InstalledAppFlow.from_client_secrets_file(
#             './data/client_secret_39562377782-nge5sdugil9eurkbgn54temjtgq06tbh.apps.googleusercontent.com.json', SCOPES)
#         creds = flow.run_local_server(port=0)

#     # token.json에 저장
#     with open('./data/token.json', 'w') as token:
#         token.write(creds.to_json())

# CalendarToolkit Setting

In [7]:
from langchain_google_community import CalendarToolkit
from langchain_google_community.calendar.utils import (
    build_resource_service,
    get_google_credentials,
)

# Can review scopes here: https://developers.google.com/calendar/api/auth
# For instance, readonly scope is https://www.googleapis.com/auth/calendar.readonly
credentials = get_google_credentials(
    token_file="./data/token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="./data/client_secret_39562377782-nge5sdugil9eurkbgn54temjtgq06tbh.apps.googleusercontent.com.json",
)

api_resource = build_resource_service(credentials=credentials)
toolkit = CalendarToolkit(api_resource=api_resource)
toolkit

CalendarToolkit(api_resource=<googleapiclient.discovery.Resource object at 0x000001B849380AD0>)

In [8]:
tools = toolkit.get_tools()
tools

[CalendarCreateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x000001B849380AD0>),
 CalendarSearchEvents(api_resource=<googleapiclient.discovery.Resource object at 0x000001B849380AD0>),
 CalendarUpdateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x000001B849380AD0>),
 GetCalendarsInfo(api_resource=<googleapiclient.discovery.Resource object at 0x000001B849380AD0>),
 CalendarMoveEvent(api_resource=<googleapiclient.discovery.Resource object at 0x000001B849380AD0>),
 CalendarDeleteEvent(api_resource=<googleapiclient.discovery.Resource object at 0x000001B849380AD0>),
 GetCurrentDatetime(api_resource=<googleapiclient.discovery.Resource object at 0x000001B849380AD0>)]

# 캘린더 목록 조회

In [9]:
def list_calendars():
    calendar_list = api_resource.calendarList().list().execute()
    items = calendar_list.get('items', [])
    
    print("캘린더 목록:")
    for calendar in items:
        print(f"- {calendar.get('summary')} (ID: {calendar.get('id')})")
res = list_calendars()
res

캘린더 목록:
- 대한민국의 휴일 (ID: ko.south_korea#holiday@group.v.calendar.google.com)
- 가족 (ID: family13100623974177768096@group.calendar.google.com)
- jongbaekim0710@gmail.com (ID: jongbaekim0710@gmail.com)


# 캘린더 이벤트 추가

In [10]:
from datetime import datetime, timedelta
def add_meeting_event(calendar_id: str, start_datetime: datetime, summary: str = "회의"):
    end_datetime = start_datetime + timedelta(hours=1)

    event = {
        "summary": summary,
        "start": {
            "dateTime": start_datetime.isoformat(),
            "timeZone": "Asia/Seoul",  # 필요 시 변경
        },
        "end": {
            "dateTime": end_datetime.isoformat(),
            "timeZone": "Asia/Seoul",
        },
    }

    created_event = api_resource.events().insert(calendarId=calendar_id, body=event).execute()
    print(f"일정이 추가되었습니다: {created_event.get('htmlLink')}")

In [16]:
start_time = datetime(2025, 5, 13, 14, 0)
start_time, type(start_time)

(datetime.datetime(2025, 5, 13, 14, 0), datetime.datetime)

In [11]:
calendar_id = "jongbaekim0710@gmail.com"  # 또는 사용자 캘린더 ID
start_time = datetime(2025, 5, 13, 14, 0)  # 2025년 5월 15일 오후 2시
add_meeting_event(calendar_id, start_time)

일정이 추가되었습니다: https://www.google.com/calendar/event?eid=dHBmM2xqdGoyaDVhcjZtYjIzOHB0YWx0djAgam9uZ2JhZWtpbTA3MTBAbQ


# 캘린더 이벤트 아이디 조회

In [12]:
def list_events(calendar_id: str, date: datetime):
    time_min = date.isoformat() + "Z"
    time_max = (date + timedelta(days=1)).isoformat() + "Z"

    events_result = api_resource.events().list(
        calendarId=calendar_id,
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy="startTime"
    ).execute()
    
    events = events_result.get("items", [])
    for event in events:
        print(f"제목: {event.get('summary')}, ID: {event.get('id')}")

# 사용 예시
from datetime import datetime, timedelta
list_events("primary", datetime(2025, 5, 13))  # 해당 날짜의 이벤트 목록 확인

제목: 2년차연수(강유), ID: 0hg7ajd63gk91g4nukf54f3sbe
제목: 워케이션(성덕), ID: 0j7mhreb1lqafkrt30d2tkivns
제목: 미래위저녁약속, ID: 13rjiqao39krafn9r3m1bv1483
제목: 삼호정기회의, ID: 3sdq9as4jmga4kac385um8n42l_20250513T000000Z
제목: 조별식사, ID: 54bqr154m643g5o968qd0tphkf
제목: 회의, ID: tpf3ljtj2h5ar6mb238ptaltv0
제목: 2년차연수(세은, 진형), ID: 2m8jhop9f786b0g6ik2ibsjpqf


# 캘린더 이벤트 삭제

In [14]:
def delete_event(calendar_id: str, event_id: str):
    try:
        api_resource.events().delete(calendarId=calendar_id, eventId=event_id).execute()
        print(f"이벤트(ID: {event_id})가 삭제되었습니다.")
    except Exception as e:
        print(f"이벤트 삭제 중 오류 발생: {e}")

In [15]:
calendar_id = "jongbaekim0710@gmail.com"  # 또는 다른 캘린더 ID
event_id = "tpf3ljtj2h5ar6mb238ptaltv0"  # 예: '5a4sk8s9d0fsdkf0sdg'
delete_event(calendar_id, event_id)

이벤트(ID: tpf3ljtj2h5ar6mb238ptaltv0)가 삭제되었습니다.
